# Negative Marking


---

Short fun script to figure out how what mark guessing would give.

## Use random number generator crate 

In [2]:
extern crate rand;
use rand::distributions::{Distribution, Uniform};
use rand::{thread_rng,Rng};
use rand::seq::SliceRandom;

## Generate a set of random responses

In [17]:
fn generate_random_answers(num_questions: usize, num_options: u8) -> Vec<u8> {
    // Initialise uniform distribution between 1 and 4
    let between = Uniform::new(1_u8, 1 + num_options);
    
    // Generate 30 "answers"
    thread_rng().sample_iter(&between).take(num_questions).collect::<Vec<u8>>()
}

In [4]:
let a = generate_random_answers(30, 4);
println!("{:?}", a);

[4, 4, 4, 1, 3, 1, 4, 4, 1, 3, 4, 4, 4, 3, 1, 4, 3, 4, 2, 4, 1, 3, 2, 1, 4, 2, 1, 3, 4, 4]


## Allocate marks

In [5]:
fn allocate_marks(num_one_mark: usize, num_two_mark: usize) -> Vec<u8> {
    let mut marks = vec![1_u8; num_one_mark];
    let twos = vec![2_u8; num_two_mark];
    
    marks.extend(twos);
    
    let mut rng = thread_rng();
    marks.shuffle(&mut rng);
    
    marks
}

In [6]:
let b = allocate_marks(20, 10);
println!("{:?}", b);

[2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1]


## Create a scenario

In [72]:
fn scenario(num_questions: usize, num_options: u8, num_one_mark: usize, 
            num_two_mark: usize, num_times: usize, penalty: f64) -> f64 
{
    // Ensure penality is positive
    let updated_penalty;
    
    if penalty < 0.0 {
        updated_penalty = -1.0*penalty
    } else {
        updated_penalty = penalty
    }
    
    // Randomly allocate correct answers and marks
    let correct_answers = generate_random_answers(num_questions, num_options);
    let mark_allocation = allocate_marks(num_one_mark, num_two_mark);
    
    // Combine answers and marks for easy comparison at next step
    let answer_and_marks = correct_answers.iter().zip(mark_allocation.iter());
    
    // If correct answer, allocate mark. Else, implement penalty
    let total_scores: f64 = (0..num_times).fold(0_f64, |scores_sum, _| 
        scores_sum + answer_and_marks.clone()
                                      .zip(generate_random_answers(num_questions, num_options).iter())
                                      .fold(0_f64, |test_sum, values| 
                                          if values.0.0 == values.1 {
                                              test_sum + *values.0.1 as f64
                                          } else {
                                              test_sum - updated_penalty
                                          })
    );
    
    total_scores/(num_times as f64)
}

In [71]:
let num_questions = 30;
let num_options = 4;
let num_one_mark = 20;
let num_two_mark = 10;
let num_times = 1000000;
let penalty = 0.5;
let test = scenario(num_questions, num_options, num_one_mark, num_two_mark, num_times, penalty);
println!("{:?}", test);



-1.24887
